# Getting started

### CLEF 2025 - CheckThat! Lab  - Task 4 Scientific Web Discourse - Subtask 4b (Scientific Claim Source Retrieval)

This notebook enables participants of subtask 4b to quickly get started. It includes the following:
- Code to upload data, including:
    - code to upload the collection set (CORD-19 academic papers' metadata)
    - code to upload the query set (tweets with implicit references to CORD-19 papers)
- Code to run a baseline retrieval model (BM25)
- Code to evaluate the baseline model

Participants are free to use this notebook and add their own models for the competition..

# 1) Importing data

In [3]:
import numpy as np
import pandas as pd

## 1.a) Import the collection set
The collection set contains metadata of CORD-19 academic papers.

The preprocessed and filtered CORD-19 dataset is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b

Participants should first download the file then upload it on the Google Colab session with the following steps.


In [4]:
# 1) Download the collection set from the Gitlab repository: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b
# 2) Drag and drop the downloaded file to the "Files" section (left vertical menu on Colab)
# 3) Modify the path to your local file path
PATH_COLLECTION_DATA = 'subtask4b_collection_data.pkl' #MODIFY PATH

In [5]:
df_collection = pd.read_pickle(PATH_COLLECTION_DATA)

In [6]:
df_collection.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7718 entries, 162 to 1056448
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   cord_uid          7718 non-null   object        
 1   source_x          7718 non-null   object        
 2   title             7718 non-null   object        
 3   doi               7677 non-null   object        
 4   pmcid             4959 non-null   object        
 5   pubmed_id         6233 non-null   object        
 6   license           7718 non-null   object        
 7   abstract          7718 non-null   object        
 8   publish_time      7715 non-null   object        
 9   authors           7674 non-null   object        
 10  journal           6668 non-null   object        
 11  mag_id            0 non-null      float64       
 12  who_covidence_id  528 non-null    object        
 13  arxiv_id          20 non-null     object        
 14  label             7718 n

In [ ]:
df_collection.head()

## 1.b) Import the query set

The query set contains tweets with implicit references to academic papers from the collection set.

The preprocessed query set is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b

Participants should first download the file then upload it on the Google Colab session with the following steps.

In [8]:
# 1) Download the query tweets from the Gitlab repository: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b?ref_type=heads
# 2) Drag and drop the downloaded file to the "Files" section (left vertical menu on Colab)
# 3) Modify the path to your local file path
PATH_QUERY_TRAIN_DATA = 'subtask4b_query_tweets_train.tsv' #MODIFY PATH
PATH_QUERY_DEV_DATA = 'subtask4b_query_tweets_dev.tsv' #MODIFY PATH
PATH_QUERY_TEST_DATA = 'subtask4b_query_tweets_test.tsv'

In [9]:
df_query_train = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep = '\t')
df_query_dev = pd.read_csv(PATH_QUERY_DEV_DATA, sep = '\t')
df_query_test = pd.read_csv(PATH_QUERY_TEST_DATA, sep = '\t')

In [10]:
df_query_train.head()

,post_id,tweet_text,cord_uid
0,0,Oral care in rehabilitation medicine: oral vul...,htlvpvz5
1,1,this study isn't receiving sufficient attentio...,4kfl29ul
2,2,"thanks, xi jinping. a reminder that this study...",jtwb17u8
3,3,Taiwan - a population of 23 million has had ju...,0w9k8iy1
4,4,Obtaining a diagnosis of autism in lower incom...,tiqksd69


In [11]:
df_query_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12853 entries, 0 to 12852
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_id     12853 non-null  int64 
 1   tweet_text  12853 non-null  object
 2   cord_uid    12853 non-null  object
dtypes: int64(1), object(2)
memory usage: 301.4+ KB


In [12]:
df_query_dev.head()

,post_id,tweet_text,cord_uid
0,16,covid recovery: this study from the usa reveal...,3qvh482o
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu
2,73,I recall early on reading that researchers who...,sts48u9i
3,93,You know you're credible when NIH website has ...,3sr2exq9
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy


In [13]:
df_query_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_id     1400 non-null   int64 
 1   tweet_text  1400 non-null   object
 2   cord_uid    1400 non-null   object
dtypes: int64(1), object(2)
memory usage: 32.9+ KB


In [14]:
df_query_test.head()

,post_id,tweet_text
0,1,A recent research study published yesterday cl...
1,2,"""We should track the long-term effects of thes..."
2,3,"the agony of ""long haul"" covid-19 symptoms."
3,4,Home and online monitoring and assessment of b...
4,5,"it may be a long one, folks! to avoid exceedin..."


# 2) Running the baseline
The following code runs a BM25 baseline.


In [28]:
!pip install rank_bm25
from rank_bm25 import BM25Okapi

ERROR: ld.so: object '/opt/conda/lib/libmkl_def.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/opt/conda/lib/libmkl_avx2.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/opt/conda/lib/libmkl_core.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/opt/conda/lib/libmkl_intel_lp64.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/opt/conda/lib/libmkl_intel_thread.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/opt/conda/lib/libmkl_def.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/opt/conda/lib/libmkl_avx2.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/opt/conda/lib/libmkl_core.so' from LD_PRE

In [16]:
# Create the BM25 corpus
corpus = df_collection[:][['title', 'abstract']].apply(lambda x: f"{x['title']} {x['abstract']}", axis=1).tolist()
cord_uids = df_collection[:]['cord_uid'].tolist()
tokenized_corpus = [doc.split(' ') for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [17]:
def get_top_cord_uids(query):
  text2bm25top = {}
  if query in text2bm25top.keys():
      return text2bm25top[query]
  else:
      tokenized_query = query.split(' ')
      doc_scores = bm25.get_scores(tokenized_query)
      indices = np.argsort(-doc_scores)[:5]
      bm25_topk = [cord_uids[x] for x in indices]

      text2bm25top[query] = bm25_topk
      return bm25_topk


In [18]:
# Retrieve topk candidates using the BM25 model
df_query_train['bm25_topk'] = df_query_train['tweet_text'].apply(lambda x: get_top_cord_uids(x))
df_query_dev['bm25_topk'] = df_query_dev['tweet_text'].apply(lambda x: get_top_cord_uids(x))

# 3) Evaluating the baseline
The following code evaluates the BM25 retrieval baseline on the query set using the Mean Reciprocal Rank score (MRR@5).

In [19]:
# Evaluate retrieved candidates using MRR@k
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        #performances.append(data["in_topx"].mean())
        d_performance[k] = data["in_topx"].mean()
    return d_performance


In [20]:
results_train = get_performance_mrr(df_query_train, 'cord_uid', 'bm25_topk')
results_dev = get_performance_mrr(df_query_dev, 'cord_uid', 'bm25_topk')
# Printed MRR@k results in the following format: {k: MRR@k}
print(f"Results on the train set: {results_train}")
print(f"Results on the dev set: {results_dev}")

Results on the train set: {1: 0.5079747918773827, 5: 0.5508999196037242, 10: 0.5508999196037242}
Results on the dev set: {1: 0.505, 5: 0.5520357142857142, 10: 0.5520357142857142}


# 4) Exporting results to prepare the submission on Codalab

In [21]:
df_query_dev['preds'] = df_query_dev['bm25_topk'].apply(lambda x: x[:5])

In [22]:
df_query_dev[['post_id', 'preds']].to_csv('predictions.tsv', index=None, sep='\t')

## Initial Bi-Encoder Ranking with SentenceTransformer

In [ ]:
!pip install -U sentence-transformers
!pip install keras==2.11

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Extract text and identifiers
tweet_ids = df_query_test['post_id'].tolist()
tweet_texts = df_query_test['tweet_text'].tolist()
paper_ids = df_collection['cord_uid'].tolist()
paper_texts = df_collection['abstract'].tolist()

In [ ]:
# Load fast BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [56]:
# Compute embeddings
tweet_embeds = model.encode(tweet_texts, batch_size=64, show_progress_bar=True, convert_to_numpy=True)
paper_embeds = model.encode(paper_texts, batch_size=64, show_progress_bar=True, convert_to_numpy=True)

Batches:   0%|          | 0/23 [00:00<?, ?it/s]

Batches:   0%|          | 0/121 [00:00<?, ?it/s]

In [57]:
# Compute cosine similarity between each tweet and all papers
similarity_matrix = cosine_similarity(tweet_embeds, paper_embeds)

In [69]:
# Get top 5 paper IDs for each tweet
top_k = 5
top5_indices = np.argsort(-similarity_matrix, axis=1)[:, :top_k]
top5_cord_uids = [[paper_ids[i] for i in row] for row in top5_indices]
top5_cord_uids_str = ['[' + ', '.join(f'"{uid}"' for uid in ids) + ']' for ids in top5_cord_uids]

In [70]:
# Prepare submission DataFrame
output_df = pd.DataFrame({
    'post_id': tweet_ids,
    'preds': top5_cord_uids_str
})

# Save to TSV
output_df.to_csv('subtask4b_predictions.tsv', sep='\t', index=False)


In [71]:
output_df.head()

,post_id,preds
0,1,"[""8zufbeuz"", ""5aev7ltr"", ""x4zuv4jo"", ""26kx1b0h..."
1,2,"[""582ts5mf"", ""y8puocsa"", ""pofysmv8"", ""vw9jd88a..."
2,3,"[""gtp5daep"", ""25aj8rj5"", ""dikrttn6"", ""kb3xi0fk..."
3,4,"[""ru2ty1y9"", ""enlj85zc"", ""z43p1dmf"", ""vj5ahoi2..."
4,5,"[""0ul6k6uw"", ""lbrpbhpg"", ""3akrube4"", ""bfm45zas..."


Evaluation, which performed very poorly here, we should do further reranking on BM25 results or results from Deni/Marina, to see if reranking will help.

In [72]:
import pandas as pd
import ast

preds_df = pd.read_csv('subtask4b_predictions.tsv', sep='\t')
train_df = pd.read_csv('subtask4b_query_tweets_train.tsv', sep='\t')

# Now this will succeed
preds_df['preds'] = preds_df['preds'].apply(ast.literal_eval)

merged_df = train_df.merge(preds_df, on='post_id')

def get_performance_mrr(data, col_gold, col_pred, list_k=[1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(
            lambda x: (1 / ([i for i in x[col_pred][:k]].index(x[col_gold]) + 1)
                       if x[col_gold] in [i for i in x[col_pred][:k]] else 0),
            axis=1)
        d_performance[k] = data["in_topx"].mean()
    return d_performance

results = get_performance_mrr(merged_df, col_gold='cord_uid', col_pred='preds')
print("MRR@k results:", results)


MRR@k results: {1: 0.0, 5: 0.000501930501930502, 10: 0.000501930501930502}


## BERT-based Pairwise Scoring and Reranking

In [ ]:
!pip3 install transformers


In [75]:
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

# Load tokenizer and model with classification head (1 output)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def score_pairs(tweet, candidate_cord_uids, df_collection):
    inputs = []
    for uid in candidate_cord_uids:
        abstract = df_collection[df_collection['cord_uid'] == uid]['abstract'].values[0]
        inputs.append((tweet, abstract))

    # Tokenize all pairs
    encodings = tokenizer.batch_encode_plus(
        inputs,
        padding=True,
        truncation=True,
        return_tensors='tf'
    )

    # Predict relevance scores
    outputs = model(encodings).logits  # Shape: (batch_size, 1)
    scores = tf.squeeze(outputs, axis=1).numpy()

    return list(zip(candidate_cord_uids, scores))

In [78]:
def rerank_topk(tweet_text, candidate_uids, df_collection):
    scored = score_pairs(tweet_text, candidate_uids, df_collection)
    reranked = sorted(scored, key=lambda x: x[1], reverse=True)
    return [uid for uid, score in reranked[:5]]

## Apply reranking on actual predictions data 
(As mentioned before, we have poor predictions here, this should be applied on better predictions)

In [80]:
from tqdm import tqdm
import ast

preds_df = pd.read_csv('subtask4b_predictions.tsv', sep='\t')
preds_df['preds'] = preds_df['preds'].apply(ast.literal_eval)  # Convert to list

# Load test tweets
test_df = pd.read_csv('subtask4b_query_tweets_test.tsv', sep='\t')

# Re-rank predictions using BERT
final_preds = []

for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    post_id = row['post_id']
    tweet = row['tweet_text']

    # Get retrieved candidates
    candidate_uids = preds_df[preds_df['post_id'] == post_id]['preds'].values[0]

    # Run BERT-based re-ranking
    top5_reranked = rerank_topk(tweet, candidate_uids, df_collection)

    final_preds.append({
        'post_id': post_id,
        'preds': '[' + ', '.join(f'"{uid}"' for uid in top5_reranked) + ']'
    })


  0%|          | 0/1446 [00:00<?, ?it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  0%|          | 3/1446 [00:06<49:09,  2.04s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware

In [81]:
output_df = pd.DataFrame(final_preds)
output_df.to_csv('subtask4b_reranked_predictions.tsv', sep='\t', index=False)

In [83]:
import pandas as pd
import ast

# Load gold labels from train set
train_df = pd.read_csv('subtask4b_query_tweets_train.tsv', sep='\t')

# Load reranked predictions
preds_df = pd.read_csv('subtask4b_reranked_predictions.tsv', sep='\t')

# Convert stringified list to actual list
preds_df['preds'] = preds_df['preds'].apply(ast.literal_eval)

# Merge predictions with gold labels (on post_id)
merged_df = df_query_train.merge(preds_df, on='post_id')


In [84]:
# Evaluation function (from notebook)
def get_performance_mrr(data, col_gold, col_pred, list_k=[1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(
            lambda x: (1 / ([i for i in x[col_pred][:k]].index(x[col_gold]) + 1)
                       if x[col_gold] in x[col_pred][:k] else 0),
            axis=1)
        d_performance[k] = data["in_topx"].mean()
    return d_performance

# Run evaluation
results = get_performance_mrr(merged_df, col_gold='cord_uid', col_pred='preds')
print("MRR@k results:", results)


MRR@k results: {1: 0.0007722007722007722, 5: 0.001287001287001287, 10: 0.001287001287001287}


Since the results of previous calculations werent impressive, we decided to rerank predictions the original BM25 model produced

In [ ]:
# Retrieve topk candidates using the BM25 model
df_query_train['bm25_topk'] = df_query_train['tweet_text'].apply(lambda x: get_top_cord_uids(x))
df_query_dev['bm25_topk'] = df_query_dev['tweet_text'].apply(lambda x: get_top_cord_uids(x))